In [ ]:
##open ai

##load credentials 

from dotenv import load_dotenv
import os

load_dotenv('../cred.env')

os.environ['OPENAI_API_KEY'] = os.getenv('openai_api_keys')

#OPENAI_API_KEY= os.environ['OPENAI_API_KEY']



In [17]:
## write function to use openai instance and get findings



from openai import OpenAI

client = OpenAI()

prompt = 'write a 10 lines poem on moon and stars with each sentence on newline'

response = client.responses.create(

    model = 'gpt-4o',
    input= prompt

)

print(response.output[0].content[0].text)


In the velvet night, the moon does glide,  
A silver coin on a vast, dark tide.  
Stars like whispers, tender and bright,  
Scattered gems in the cloak of night.  

They dance and flicker in cosmic play,  
Guiding lost souls who’ve lost their way.  
Dreams are woven in their gentle glow,  
Secrets the heavens quietly know.  

Silent guardians of night’s embrace,  
Eternal beauty in endless space.  


In [21]:
## write with role and message 

from openai import OpenAI

client = OpenAI()

sys_msg = 'You are poem writer for toddler books'
usr_prompt = 'write a 10 lines poem on moon and stars with each sentence on newline'


reponse = client.chat.completions.create(

model = 'gpt-4o',
messages = [

    {'role':'system', 'content':sys_msg},
    {'role':'user','content':usr_prompt}

],

temperature= 1

)

print(reponse.choices[0].message.content)

The moon peeks out, a glowing face,  
Floating gently in the sky's embrace.  
Silver beams on a gentle flight,  
Guide the stars through the night.  

Twinkling dots in a velvet sea,  
Sparkling brightly, wild and free.  
Like diamonds in the dark so wide,  
They dance and play in cosmic tide.  

Moon and stars with magic gleam,  
Whisper dreams in sleep's sweet stream.  


In [22]:
## for image

from openai import OpenAI

client = OpenAI()

sys_msg = 'Write concise summary of image'
usr_prompt = 'write a 10 lines poem on moon and stars with each sentence on newline'
image_url ='https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI'

reponse = client.chat.completions.create(

model = 'gpt-4o',
messages = [

    {'role':'system', 'content':sys_msg},

    {'role':'user','content':[

        {'type':'image_url','image_url':{'url': image_url}}

    ]
    }

],

temperature= 1

)

print(reponse.choices[0].message.content)

A snow-covered mountain peak rises against a backdrop of a pastel-colored sky at sunrise or sunset. The scene is tranquil and picturesque, showcasing the beauty of nature.


1.	Write a Python function to upsert raw texts into Pinecone without LangChain.

In [ ]:
##Write a Python function to upsert raw texts into Pinecone without LangChain.

from openai import OpenAI
from openai_pinecone import OpenAIPinecone
from dot_env import load_dotenv

load_dotenv(dotenv_path="../cred.env")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

##create index 

pc = OpenAIPinecone(api_key=OPENAI_API_KEY, env='us-west1-gcp')

index = pc.create_index(
    name="test",
    dimension=1536,
    metric="cosine",
    pod_type="p1"


)

pc.upsert(
    namespace="test",
    text="hello world",
    id="1"
)

In [ ]:
##2.	Modify your function to include metadata (like filename, chunk index).


pc.upsert(
    namespace="test",
    text="hello world",
    id="1",
    metadata={"filename": "file1.txt", "chunk_index": 1}
)

In [ ]:
#3.	Implement a search function that returns the top 3 most similar texts.

index.query(
    namespace="test",
    query="hello world",
    top_k=3,
    include_values=True,
    include_metadata=True
)

In [ ]:
## 5.	Show how to delete or update vectors in an existing Pinecone index.


index.delete(id="1", namespace="test") ## here namespace is mandatory

index.update(
    id="1",
    namespace="test",
    text="hello world 123",
    metadata={"filename": "file1.txt", "chunk_index": 1}
)

In [1]:
mys = ('abc',40)

print(type(mys))

<class 'tuple'>


In [3]:
sqr = lambda x=10: x ** 2

print(sqr(5))

print(sqr())

25
100


In [5]:
## encoding with tiktoken
import tiktoken 
import pandas as pd

df = pd.read_csv('words.csv')


encoding = tiktoken.encoding_for_model("gpt-4")



In [6]:
df.head(1)

,columnA,columnB,columnC,columnD,columnE,columnF
0,stems,tiny,bus,wore,next,planet


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   columnA  7 non-null      object
 1   columnB  7 non-null      object
 2   columnC  7 non-null      object
 3   columnD  7 non-null      object
 4   columnE  7 non-null      object
 5   columnF  7 non-null      object
dtypes: object(6)
memory usage: 468.0+ bytes


In [8]:
df

,columnA,columnB,columnC,columnD,columnE,columnF
0,stems,tiny,bus,wore,next,planet
1,planet,tiny,planet,swam,managed,respect
2,southern,time,since,gently,copper,doll
3,perhaps,dull,trip,tiny,perhaps,attention
4,tiny,person,planet,product,myself,tiny
5,browserling,instance,minerals,state,inch,next
6,planet,nor,studied,progress,ear,planet


In [9]:

## create embeddings for dataframe

from dotenv import load_dotenv
from openai import OpenAI
import os 

load_dotenv('../cred.env')

import openai

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

def get_embeddings(text, model='text-embedding-3-small' ):
    client = OpenAI()
    text = text.replace('\\n',' ')

    response = client.embeddings.create(
        input = text,
        model = model,
        dimensions=1536  
    )

    return response.data[0].embedding






In [10]:
for column in df.columns:
    print(column)
    df[column + '_embeddings'] = df[column].apply(lambda x: get_embeddings(x))

columnA
columnB
columnC
columnD
columnE
columnF


In [11]:
len(df.columnA_embeddings[0])

1536

In [15]:
##get embedding cost with tiktoken

import tiktoken

def get_embedding_cost (text):

    enc=tiktoken.encoding_for_model('text-embedding-3-small')

    #toke_count =  sum([len(enc.encode(page.page_content)) for page in texts])

    token_count = len(enc.encode(text))

    print(f'Total Tokens: {token_count}')

    return (token_count/1000) * 0.00002 # token_count/1000 * 0.00002

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   columnA             7 non-null      object
 1   columnB             7 non-null      object
 2   columnC             7 non-null      object
 3   columnD             7 non-null      object
 4   columnE             7 non-null      object
 5   columnF             7 non-null      object
 6   columnA_embeddings  7 non-null      object
 7   columnB_embeddings  7 non-null      object
 8   columnC_embeddings  7 non-null      object
 9   columnD_embeddings  7 non-null      object
 10  columnE_embeddings  7 non-null      object
 11  columnF_embeddings  7 non-null      object
dtypes: object(12)
memory usage: 804.0+ bytes


In [50]:
df_text_columns = ~df.columns.str.contains('embedding')

df_text =df.loc[:,df_text_columns]

In [13]:
df_text

,columnA,columnB,columnC,columnD,columnE,columnF
0,stems,tiny,bus,wore,next,planet
1,planet,tiny,planet,swam,managed,respect
2,southern,time,since,gently,copper,doll
3,perhaps,dull,trip,tiny,perhaps,attention
4,tiny,person,planet,product,myself,tiny
5,browserling,instance,minerals,state,inch,next
6,planet,nor,studied,progress,ear,planet


In [17]:
df_text.applymap(get_embedding_cost).sum().sum()

Total Tokens: 2
Total Tokens: 1
Total Tokens: 3
Total Tokens: 1
Total Tokens: 1
Total Tokens: 2
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 2
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 2
Total Tokens: 2
Total Tokens: 2
Total Tokens: 2
Total Tokens: 2
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 2
Total Tokens: 1
Total Tokens: 2
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1
Total Tokens: 1


C:\Users\prach\AppData\Local\Temp\ipykernel_25940\3125867080.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_text.applymap(get_embedding_cost).sum().sum()


np.float64(1.0799999999999998e-06)

In [57]:
## serach perticular word from dataframe

word = 'tiny'

word_embedding = get_embeddings(word)

word_embedding

[-0.005895537324249744,
 -0.02821900136768818,
 -0.004158630035817623,
 -0.018578018993139267,
 -0.04095121845602989,
 -0.019220752641558647,
 0.012403200380504131,
 0.06188592314720154,
 -0.014140107668936253,
 -0.008240744471549988,
 -0.005796066485345364,
 0.01586936227977276,
 -0.03923726826906204,
 0.02577049843966961,
 0.010551519691944122,
 -0.007693656720221043,
 -0.018822869285941124,
 -0.03013189509510994,
 0.038992416113615036,
 -0.010168940760195255,
 0.003837263910099864,
 0.0027756080962717533,
 0.042542748153209686,
 0.011454405263066292,
 0.014147758483886719,
 0.028555670753121376,
 0.025632770732045174,
 -0.012449109926819801,
 0.05989651381969452,
 -0.031585693359375,
 0.009396132081747055,
 -0.03801301494240761,
 0.01605300046503544,
 -0.05056159570813179,
 0.005796066485345364,
 0.03155508637428284,
 0.0433691181242466,
 0.009924090467393398,
 0.005126554053276777,
 -0.0007221171981655061,
 -0.01687937043607235,
 -0.032442670315504074,
 0.011155993677675724,
 0.006

In [40]:
import numpy as np

def cosine_similarity_numpy(vector_x, vector_y):
    """
    Calculates the cosine similarity between two vectors using NumPy.
    
    Args:
        vector_x (list or np.array): The first vector.
        vector_y (list or np.array): The second vector.

    Returns:
        float: The cosine similarity between vector_x and vector_y.
    """
    # Convert inputs to numpy arrays if they aren't already
    A = np.array(vector_x)
    B = np.array(vector_y)
    
    # Calculate the dot product (A · B)
    dot_product = np.dot(A, B)
    
    # Calculate the magnitudes (||A|| * ||B||)
    # np.linalg.norm computes the Euclidean norm (magnitude)
    magnitude_a = np.linalg.norm(A)
    magnitude_b = np.linalg.norm(B)
    
    if magnitude_a == 0 or magnitude_b == 0:
        return 0 # Return 0 if either vector has zero magnitude (undefined similarity)
        
    # Calculate cosine similarity: (A · B) / (||A|| * ||B||)
    cosine_sim = dot_product / (magnitude_a * magnitude_b)
    
    return cosine_sim

In [54]:
df.iloc[:,6:]

,columnA_embeddings,columnB_embeddings,columnC_embeddings,columnD_embeddings,columnE_embeddings,columnF_embeddings
0,"[0.03770485520362854, 0.0154457101598382, 0.06...","[-0.005895537324249744, -0.02821900136768818, ...","[-0.018057024106383324, -0.0018465706380084157...","[0.002562168752774596, -0.004380596801638603, ...","[-0.005242754239588976, 0.015545335598289967, ...","[0.0016872665146365762, -0.01349813211709261, ..."
1,"[0.0016872665146365762, -0.01349813211709261, ...","[-0.005896334070712328, -0.028222816064953804,...","[0.0017549099866300821, -0.013497919775545597,...","[-0.01043359749019146, 0.03809491917490959, -0...","[0.06400150060653687, 0.03609146177768707, 0.0...","[0.07422041147947311, -0.022316085174679756, -..."
2,"[0.001222960534505546, -0.019330903887748718, ...","[-0.02206721156835556, -0.019183265045285225, ...","[-0.023011287674307823, -0.034682147204875946,...","[0.04720812290906906, 0.0008303297217935324, -...","[0.003881772980093956, -0.06735920161008835, -...","[0.026863712817430496, -0.0403079017996788, -0..."
3,"[-0.02073061466217041, -0.03273814916610718, 0...","[-0.014250528998672962, -0.008181098848581314,...","[0.0033266947139054537, -0.0004146453284192830...","[-0.0058506447821855545, -0.028223907575011253...","[-0.02073061466217041, -0.03273814916610718, 0...","[0.0835820883512497, -0.001664588926360011, -0..."
4,"[-0.005895537324249744, -0.02821900136768818, ...","[0.02762443758547306, -0.04298079013824463, -0...","[0.0017207404598593712, -0.013501194305717945,...","[-0.0021799462847411633, -0.007144792936742306...","[0.033398739993572235, -0.05387407913804054, 0...","[-0.005895537324249744, -0.02821900136768818, ..."
5,"[-0.02954678423702717, 0.01114508043974638, 0....","[-0.00925473403185606, -0.00871199555695057, -...","[-0.022444909438490868, 0.03200390934944153, 0...","[-0.018071576952934265, -0.005310569889843464,...","[0.031539808958768845, -0.028438441455364227, ...","[-0.005242754239588976, 0.015545335598289967, ..."
6,"[0.0016872665146365762, -0.01349813211709261, ...","[-0.030872942879796028, -0.004317590966820717,...","[-0.004893322940915823, -0.027588341385126114,...","[0.03138451278209686, 0.012810156680643559, 0....","[-0.0002706248196773231, -0.019589418545365334...","[0.0016739896964281797, -0.013536305166780949,..."


In [58]:
for columns in df.iloc[:,6:].columns:
    print(columns)
    df[columns + 'simlarity'] = df[columns].apply(lambda x: cosine_similarity_numpy(x,word_embedding))

columnA_embeddings
columnB_embeddings
columnC_embeddings
columnD_embeddings
columnE_embeddings
columnF_embeddings


In [61]:
df

,columnA,columnB,columnC,columnD,columnE,columnF,columnA_embeddings,columnB_embeddings,columnC_embeddings,columnD_embeddings,columnE_embeddings,columnF_embeddings,columnA_embeddingssimlarity,columnB_embeddingssimlarity,columnC_embeddingssimlarity,columnD_embeddingssimlarity,columnE_embeddingssimlarity,columnF_embeddingssimlarity
0,stems,tiny,bus,wore,next,planet,"[0.03770485520362854, 0.0154457101598382, 0.06...","[-0.005895537324249744, -0.02821900136768818, ...","[-0.018057024106383324, -0.0018465706380084157...","[0.002562168752774596, -0.004380596801638603, ...","[-0.005242754239588976, 0.015545335598289967, ...","[0.0016872665146365762, -0.01349813211709261, ...",0.260203,1.000000,0.335740,0.256795,0.308763,0.296996
1,planet,tiny,planet,swam,managed,respect,"[0.0016872665146365762, -0.01349813211709261, ...","[-0.005896334070712328, -0.028222816064953804,...","[0.0017549099866300821, -0.013497919775545597,...","[-0.01043359749019146, 0.03809491917490959, -0...","[0.06400150060653687, 0.03609146177768707, 0.0...","[0.07422041147947311, -0.022316085174679756, -...",0.296996,0.999999,0.296916,0.199076,0.321965,0.277170
2,southern,time,since,gently,copper,doll,"[0.001222960534505546, -0.019330903887748718, ...","[-0.02206721156835556, -0.019183265045285225, ...","[-0.023011287674307823, -0.034682147204875946,...","[0.04720812290906906, 0.0008303297217935324, -...","[0.003881772980093956, -0.06735920161008835, -...","[0.026863712817430496, -0.0403079017996788, -0...",0.270145,0.417774,0.255429,0.403873,0.250646,0.451971
3,perhaps,dull,trip,tiny,perhaps,attention,"[-0.02073061466217041, -0.03273814916610718, 0...","[-0.014250528998672962, -0.008181098848581314,...","[0.0033266947139054537, -0.0004146453284192830...","[-0.0058506447821855545, -0.028223907575011253...","[-0.02073061466217041, -0.03273814916610718, 0...","[0.0835820883512497, -0.001664588926360011, -0...",0.398013,0.289507,0.364804,0.999999,0.398013,0.327845
4,tiny,person,planet,product,myself,tiny,"[-0.005895537324249744, -0.02821900136768818, ...","[0.02762443758547306, -0.04298079013824463, -0...","[0.0017207404598593712, -0.013501194305717945,...","[-0.0021799462847411633, -0.007144792936742306...","[0.033398739993572235, -0.05387407913804054, 0...","[-0.005895537324249744, -0.02821900136768818, ...",1.000000,0.400059,0.296933,0.225105,0.285545,1.000000
5,browserling,instance,minerals,state,inch,next,"[-0.02954678423702717, 0.01114508043974638, 0....","[-0.00925473403185606, -0.00871199555695057, -...","[-0.022444909438490868, 0.03200390934944153, 0...","[-0.018071576952934265, -0.005310569889843464,...","[0.031539808958768845, -0.028438441455364227, ...","[-0.005242754239588976, 0.015545335598289967, ...",0.307222,0.389091,0.240718,0.262719,0.468745,0.308763
6,planet,nor,studied,progress,ear,planet,"[0.0016872665146365762, -0.01349813211709261, ...","[-0.030872942879796028, -0.004317590966820717,...","[-0.004893322940915823, -0.027588341385126114,...","[0.03138451278209686, 0.012810156680643559, 0....","[-0.0002706248196773231, -0.019589418545365334...","[0.0016739896964281797, -0.013536305166780949,...",0.296996,0.302435,0.261010,0.306603,0.302972,0.296938


In [62]:
#(Beginner) Implement should_stop(state, last_output) to check MAX_STEPS and route == "final_answer".


def should_stop(state,last_output):

    MAX_STEPS = 5

    if isinstance(last_output, dict) and last_output['tool'] == 'final_answer':
        return True

    if len(state.get('intermediate_steps', [])) >= MAX_STEPS:
        return True

    return False

In [ ]:
##Implement llm_choose_next(state, tool_list) that formats a prompt, queries the LLM, parses JSON output robustly and falls back gracefully. Add unit tests.

def llm_choose_next(state, tool_list):
    '''This function
    1. formats a prompt
    2. queries the LLM
    3. parses JSON output robustly
    4. falls back gracefully
    
     '''

     ## get recent tool_list

In [1]:
#Write an LLMChain that creates email subject lines.

from dotenv import load_dotenv
from openai import OpenAI
import os 

load_dotenv('../cred.env')

import openai

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



from langchain_classic.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI


llm = OpenAI(temperature=0.9)

prompt = PromptTemplate (
    input_variable = '{content}',
    template = 'Write a subject line for this email: {content}'
)


llm_chain = LLMChain(
    llm = llm,
    prompt = prompt)

llm_chain.invoke({'content': 'A classic example of a deterministic heuristic is the greedy algorithm for certain problems, such as making change with standard currency denominations. At each step, it makes the locally optimal choice (e.g., picking the largest possible coin value that is less than the remaining amount owed). This process is entirely deterministic; anyone following the same steps with the same initial amount will always end up with the same set of coins'})

C:\Users\prach\AppData\Local\Temp\ipykernel_35208\4174843306.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


{'content': 'A classic example of a deterministic heuristic is the greedy algorithm for certain problems, such as making change with standard currency denominations. At each step, it makes the locally optimal choice (e.g., picking the largest possible coin value that is less than the remaining amount owed). This process is entirely deterministic; anyone following the same steps with the same initial amount will always end up with the same set of coins',
 'text': '\n\n"Learn About the Deterministic Heuristic and Greedy Algorithm for Currency Change"'}

In [41]:
#Modify ConversationalRetrievalChain to return sources + confidence scores for each output.


from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import OpenAI
from langchain_classic.embeddings.openai import OpenAIEmbeddings

llm = OpenAI(temperature=0.9)


memory = ConversationBufferWindowMemory(
    k=5, return_messages=True, memory_key="chat_history", output_key='answer'
)


## get vector_store for retrriving

from langchain_pinecone import PineconeVectorStore

embeddings = OpenAIEmbeddings(model='text-embedding-3-small',dimensions=1536)

vector_store = PineconeVectorStore.from_existing_index(index_name='langgraph-research-agent',embedding=embeddings)

#retriever = vector_store.as_retriever(search_kwargs={"k": 3})

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 3,
        # Only return documents with a score above 0.7 (adjust this threshold)
        "score_threshold": 0.5
    }
)



c:\Users\prach\anaconda3\Lib\site-packages\langchain_community\embeddings\openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


In [42]:
chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory, return_source_documents=True, output_key='answer'
)

# result = chain.invoke({'question': 'what is GenAI?'})

# print (result['answer'])

help(chain)

Help on ConversationalRetrievalChain in module langchain_classic.chains.conversational_retrieval.base object:

class ConversationalRetrievalChain(BaseConversationalRetrievalChain)
 |  ConversationalRetrievalChain(
 |      *args: Any,
 |      name: str | None = None,
 |      memory: langchain_classic.base_memory.BaseMemory | None = None,
 |      callbacks: list[langchain_core.callbacks.base.BaseCallbackHandler] | langchain_core.callbacks.base.BaseCallbackManager | None = None,
 |      verbose: bool = <factory>,
 |      tags: list[str] | None = None,
 |      metadata: dict[str, typing.Any] | None = None,
 |      callback_manager: langchain_core.callbacks.base.BaseCallbackManager | None = None,
 |      combine_docs_chain: langchain_classic.chains.combine_documents.base.BaseCombineDocumentsChain,
 |      question_generator: langchain_classic.chains.llm.LLMChain,
 |      output_key: str = 'answer',
 |      rephrase_question: bool = True,
 |      return_source_documents: bool = False,
 |    

In [ ]:
from langchain_classic.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_openai import OpenAI
from langchain_classic.embeddings.openai import OpenAIEmbeddings

llm = OpenAI(temperature=0.9)

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)
## get vector_store for retrriving
import chromadb
embeddings = OpenAIEmbeddings(model='text-embedding-3-small',dimensions=1536)

vector_store = chromadb.PersistentClient(
    path="./chroma_data"
).get_collection(
    name="retirement-genai-advisor",
    embedding_function=embeddings.embed_query
)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

chain = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory, return_source_documents=True, output_key='answer'
)

result = chain.invoke({'question': 'what is GenAI?'})

In [44]:
result = chain.invoke({'question':'what is  GenAI?'})

print (result['answer'])

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
No relevant docs were retrieved using the relevance score threshold 0.5



GenAI is an artificial intelligence platform that uses advanced algorithms and data analysis to generate insights and solutions for various industries. It can process large amounts of data and make predictions and recommendations based on that data.


In [45]:
source_data = result.get('source_documents', [])

print (source_data)

[]


##RAG - Q&A App on Your Private Documents (Pinecone and Chroma) --pseudo steps

1.document loader pypdf.

2.chunk it using recursivespliter

3.create embeddings using openai - passs chunks to this


4.connect to pinecode db

5.create index

6.index.upsert pass chunks to this 

7.create a chatbuffermemory, chatconversation openai thread

In [47]:
#document loader pypdf

def load_pdf(file_path):
    from langchain_classic.document_loaders import PyPDFLoader
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

In [ ]:
documents = load_pdf('sample.pdf')

print (documents[0].page_content)

In [49]:
#chunk it using recursivespliter
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

def chunk_data(text_data, chunk_size= 500, chunk_overlap=50):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(text_data)
    return chunks



In [52]:
#create embeddings using openai - passs chunks to this function

# from langchain_openai import OpenAIEmbeddings

# def get_embeddings(chunks):

#     embeddings = OpenAIEmbeddings(model='text-embedding-3-small',dimensions=1536)
#     return embeddings


from openai import OpenAI

def get_embeddings ( chunks, model='text-embedding-3-small'):

    client = OpenAI()
    response  = client.embeddings.create(
        input = chunks,
        model = model
    )
    return response.data[0].embedding

In [58]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../cred.env')

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')



In [59]:
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=PINECONE_API_KEY,
    environment='us-east1-aws'
)

index_name = 'langchain-retrieval-augmentation'


# we create a new index
pc.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536 ,
        spec =ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
)

{
    "name": "langchain-retrieval-augmentation",
    "metric": "cosine",
    "host": "langchain-retrieval-augmentation-huqjqmm.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
#create vectorstore pass chunks to this 

from langchain_pinecone import PineconeVectorStore

def create_vectorstore (chunks, index_name='langchain-retrieval-augmentation'):

    embeddings = get_embeddings(chunks)
    vector_store = PineconeVectorStore.from_texts(chunks, embeddings, index_name=index_name)
    return vector_store

In [62]:
## chroma db, create persistent vectorstore

from langchain_classic.vectorstores import Chroma

def load_vectorstore (chunks, index_name='langchain-retrieval-augmentation'):

    embeddings = get_embeddings(chunks)
    vector_store = Chroma.from_texts(chunks, embeddings, index_name=index_name)
    return vector_store


def get_vetorstore (persist_dirctory='./chroma_db' ):
    from langchain_classic.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings(model='text-embedding-3-small',dimensions=1536)
    vector_store = Chroma(persist_directory=persist_dirctory, embedding_function=embeddings)
    return vector_store
